In [ ]:
from scripts.model_utils import Utils
import torch
from transformers import RobertaForTokenClassification, RobertaTokenizerFast, TrainingArguments, Trainer
from scripts.Reader import obtain_dataset, obtain_label_list

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    device = torch.device('cuda')
print("Current Device:", torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device()))

In [ ]:
# Load tokenizer and model
label_list, label2id, id2label = obtain_label_list("TempEval3")
model_name = 'roberta-base'
tokenizer = RobertaTokenizerFast.from_pretrained(model_name, add_prefix_space=True)
model = RobertaForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), label2id=label2id, id2label=id2label)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results/Timex3-NER",
    logging_dir="./logs/Timex3-NER",
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=100,
    num_train_epochs=1,
    save_total_limit=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [ ]:
utils = Utils(tokenizer, label_list)
tokenized_datasets = utils.tokenize_datasets(obtain_dataset("TempEval3"))

In [ ]:
tokenized_datasets

In [ ]:
timex3_ner = Trainer(
    model=model,
    args=training_args,
    compute_metrics=utils.compute_metrics,
    data_collator=utils.data_collator,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
)

In [ ]:
model.to(device)
timex3_ner.train()

In [ ]:
timex3_ner.evaluate(tokenized_datasets["test"])

In [ ]:
timex3_ner.save_model("./results/Timex3-NER/final_model")